In [1]:
import os
from glob import glob
import copy
from datetime import datetime
import distutils.dir_util
from shutil import copyfile
import ujson as json
import PIL


from matplotlib import pyplot as plt
%matplotlib inline
import bcolz
import numpy as np

from keras.utils.np_utils import to_categorical as one_hot

from keras.layers import Dense, Input, Lambda, BatchNormalization, Conv2D, ZeroPadding2D, MaxPooling2D, AveragePooling2D, Activation, Flatten, Dropout
from keras.models import Model
from keras.applications.resnet50 import ResNet50, identity_block, conv_block
from keras.applications.imagenet_utils import preprocess_input
from keras.regularizers import l2

from keras.optimizers import Adam
from keras import backend as K

from keras.callbacks import CSVLogger
from keras.preprocessing.image import ImageDataGenerator

from keras import __version__
print(__version__)

Using TensorFlow backend.


2.0.4


In [2]:
DATA_ROOT_DIR = "/home/ubuntu/data/the-nature-conservancy-fisheries-monitoring/"
TEST_STG1_DIR = DATA_ROOT_DIR + '/test_stg1/'
TEST_STG2_DIR = DATA_ROOT_DIR + '/test_stg2/'
RESULTS_DIR = DATA_ROOT_DIR + '/results/'

SAMPLE_ROOT_DIR = DATA_ROOT_DIR + "/sample/"
TRAIN_SAMPLE_DIR = SAMPLE_ROOT_DIR + '/train/'
VALID_SAMPLE_DIR = SAMPLE_ROOT_DIR + '/valid/'
TRAIN_DIR = DATA_ROOT_DIR + '/train/'
VALID_DIR = DATA_ROOT_DIR + '/valid/'

# 1. Prepare Data (create validation/sample sets, rearrange images by class dir)

See the-nature-conservancy-fisheries-monitoring_v1-keras2.ipynb

# 2. Finetune model on full data (not sample as small data)

In [3]:
def fit_generator_helper(model, result_dir_name, batch_size=32, lr=0.1, nb_epoch=1):
    K.set_value(model.optimizer.lr, lr)
    
    now = datetime.now().strftime("%Y%m%d_%H%M%S.h5")
    results_dir = RESULTS_DIR + "/" + result_dir_name + "/"
    distutils.dir_util.mkpath(results_dir)
    
    model.fit_generator(trn_batches,
                        samples_per_epoch=trn_batches.nb_sample,
                        nb_epoch=nb_epoch,
                        validation_data=val_batches,
                        nb_val_samples=val_batches.nb_sample,
                        callbacks=[CSVLogger(results_dir+"epoch_results.csv", separator=',', append=True)])
    model.save_weights(results_dir + now )
    return model

def fit_precomputed_helper(model, result_dir_name, batch_size=32, lr=0.1, nb_epoch=1):  
    K.set_value(model.optimizer.lr, lr)
    
    now = datetime.now().strftime("%Y%m%d_%H%M%S.h5")
    results_dir = RESULTS_DIR + "/" + result_dir_name + "/"
    distutils.dir_util.mkpath(results_dir)
    
    model.fit(trn_conv_features, trn_labels,
              batch_size=batch_size, 
              nb_epoch=nb_epoch,
              validation_data=(val_conv_features, val_labels),
              shuffle=True, 
              callbacks=[CSVLogger(results_dir+"epoch_results.csv", separator=',', append=True)])
    model.save_weights(results_dir + now)
    return model

def predict_generator_helper(model, batches, batch_size, num_batch_samples):
    # Be sure get_batches has used shuffle=False
    return model.predict_generator(batches,
                                   num_batch_samples // batch_size)
                                   #num_batch_samples)

def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()

In [4]:
# Create base model
resnet_base = ResNet50(include_top=False, weights='imagenet')

#classifier_input_shape = resnet_base.layers[-1].output_shape[1:] # i.e. shape of conv features (produces (None, None, None, 2048))
classifier_input_shape = (1, 1, 2048)
classifier_input = Input(shape=classifier_input_shape)

### Precompute convolutional output (to save training time)

### Run finetuning - classifier_model_v2

In [5]:
def load_array(fname):
    return bcolz.open(fname)[:]

def load_precomputed_data(features_base_name="ResNet50_conv_feats/trn_"):
    filenames = load_array(RESULTS_DIR+"/"+features_base_name+'filenames.dat').tolist()
    conv_feats = load_array(RESULTS_DIR+"/"+features_base_name+'conv_feats.dat')
    labels = load_array(RESULTS_DIR+"/"+features_base_name+'labels.dat')
    return filenames, conv_feats, labels
                          
trn_filenames, trn_conv_features, trn_labels = load_precomputed_data("ResNet50_conv_feats/trn_")
val_filenames, val_conv_features, val_labels = load_precomputed_data("ResNet50_conv_feats/val_")

In [6]:
assert len(trn_filenames) == 3397, "trn_filenames not as expected"
assert trn_conv_features.shape == (3397, 1, 1, 2048), "trn_conv_features not as expected"
assert trn_labels.shape == (3397, 8), "trn_labels not as expected"

assert len(val_filenames) == 380, "val_filenames not as expected"
assert val_conv_features.shape == (380, 1, 1, 2048), "val_conv_features not as expected"
assert val_labels.shape == (380, 8), "val_labels not as expected"

>"Intially overfit befor worrying about reducing it (helping to ensure you have a model complex enough for the orignal data)."

In [7]:
# Create classifier model
# (base on `In [23]` https://github.com/asmith26/courses/blob/master/deeplearning1/nbs/lesson7.ipynb)

p = 0.6

x = Flatten()(classifier_input)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(p)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(p/2)(x)
x = Dense(8, activation='softmax')(x)
                                                     
classifier_model_v2 = Model(classifier_input, x)

classifier_model_v2.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [59]:
classifier_model_v2 = fit_precomputed_helper(classifier_model_v2, "classifier_model_v2", lr=0.01, nb_epoch=4)
# Ran n times

Train on 3397 samples, validate on 380 samples
Epoch 1/4
 160/3397 [>.............................] - ETA: 3s - loss: 0.4668 - acc: 0.8875

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:29: UserWarning:

The `nb_epoch` argument in `fit` has been renamed `epochs`.



3397/3397 [==============================] - 3s - loss: 0.3821 - acc: 0.8681 - val_loss: 0.2702 - val_acc: 0.9211

In [60]:
classifier_model_v2 = fit_precomputed_helper(classifier_model_v2, "classifier_model_v2", lr=0.0001, nb_epoch=4)
# Ran n times

Train on 3397 samples, validate on 380 samples
Epoch 1/4
 160/3397 [>.............................] - ETA: 3s - loss: 0.2683 - acc: 0.9125

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:29: UserWarning:

The `nb_epoch` argument in `fit` has been renamed `epochs`.



3397/3397 [==============================] - 3s - loss: 0.3087 - acc: 0.8890 - val_loss: 0.2562 - val_acc: 0.9184

Visualise validation predictions with confusion matrix.

In [16]:
def create_prediction_actual_mappings(filenames, labels):
    class_mapping = ["ALB","BET","DOL","LAG","NoF","OTHER","SHARK","YFT"]

    prediction_actual_mappings = {"ALB": {
        "ALB": [], "BET":[], "DOL": [],"LAG": [],"NoF": [],"OTHER": [],"SHARK": [],"YFT": []
    },"BET":{
        "ALB": [], "BET":[], "DOL": [],"LAG": [],"NoF": [],"OTHER": [],"SHARK": [],"YFT": []
    }, "DOL": {
        "ALB": [], "BET":[], "DOL": [],"LAG": [],"NoF": [],"OTHER": [],"SHARK": [],"YFT": []
    },"LAG": {
        "ALB": [], "BET":[], "DOL": [],"LAG": [],"NoF": [],"OTHER": [],"SHARK": [],"YFT": []
    },"NoF": {
        "ALB": [], "BET":[], "DOL": [],"LAG": [],"NoF": [],"OTHER": [],"SHARK": [],"YFT": []
    },"OTHER": {
        "ALB": [], "BET":[], "DOL": [],"LAG": [],"NoF": [],"OTHER": [],"SHARK": [],"YFT": []
    },"SHARK": {
        "ALB": [], "BET":[], "DOL": [],"LAG": [],"NoF": [],"OTHER": [],"SHARK": [],"YFT": []    
    },"YFT": {
        "ALB": [], "BET":[], "DOL": [],"LAG": [],"NoF": [],"OTHER": [],"SHARK": [],"YFT": []
        }}

    idx = 0
    for file in filenames:
        actual_class = file.split("/")[0]
        filename = file.split("/")[1]

        #print(idx)
        predict_idx = np.argmax(labels[idx])
        #print(predict_idx)
        predict_class = class_mapping[predict_idx]
        #print(predict_class)

        prediction_actual_mappings[predict_class][actual_class].append(filename)

        idx += 1

    return prediction_actual_mappings

def prepare_prediction_actual_mappings_for_confusion(prediction_actual_mappings):
    x_and_y_labels = []
    counts = []

    for predicted_class, actual_filenames in sorted(prediction_actual_mappings.items()):
        x_and_y_labels.append(predicted_class)

        row_counts = []
        for actual_class, filenames in sorted(actual_filenames.items()):
            # for filename in filenames:
            row_counts.append(len(filenames))
        counts.append(row_counts)

    # Transpose to follow standard of "Actual" on side axis, "Predicted" on top
    counts = np.matrix(counts).transpose().tolist()
    return counts, x_and_y_labels

In [25]:
# import numpy as np
# from plotly.offline import plot
# import plotly.graph_objs as go

from plotly import __version__
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True) # connect to notebook
import plotly.graph_objs as go

def show_confusion_matrix(counts, x_and_y_labels, main_title="Confusion Matrix"):

    # Plot
    colorscale = [[0, '#deebf7'], [1, '#3182bd']]  # custom colorscale
    # colorscale = [[0, '#ffeda0'], [1, '#f03b20']]  # custom colorscale

    trace = go.Heatmap(
        z=counts,
        x=x_and_y_labels,
        y=x_and_y_labels,
        colorscale=colorscale,
        showscale=True
    )

    # Add text labels to each grid cell
    annotations = []
    z = counts
    x = x_and_y_labels
    y = x_and_y_labels
    for n, row in enumerate(z):
        for m, val in enumerate(row):
            #var = z[n][m]
            annotations.append(
                dict(
                    hovertext="TODO: Add list of files here (corresponding to hover box)",
                    text=str(val),
                    x=x[m], y=y[n],
                    xref='x1', yref='y1',
                    font=dict(color='white' if val > 0.5 else 'black'),
                    showarrow=False)
            )

    # Create firgure from text labels and heatmap
    fig = go.Figure(data=[trace])
    fig['layout'].update(
        title=main_title,
        annotations=annotations,
        xaxis=dict(ticks='', side='bottom', title='Predicted'),
        # ticksuffix is a workaround to add a bit of padding
        yaxis=dict(ticks='', ticksuffix='  ', title='Actual', ),
        width=700,
        height=700,
        autosize=False
    )

    iplot(fig, show_link=False)

In [33]:
def create_confusion_matrix_helper(model):
    predictions = model.predict(val_conv_features, batch_size=38)

    prediction_actual_mappings = create_prediction_actual_mappings(val_filenames, predictions)
    
    counts, x_and_y_labels = prepare_prediction_actual_mappings_for_confusion(prediction_actual_mappings)
    
    show_confusion_matrix(counts, x_and_y_labels)
    
    return prediction_actual_mappings

In [35]:
prediction_actual_mappings_base = create_confusion_matrix_helper(classifier_model_v2)

Let's have a look at those image predicted incorectly as YFT (can do within Jupyter for the directory listing):

In [36]:
prediction_actual_mappings_base

{'ALB': {'ALB': ['img_02201.jpg',
   'img_05031.jpg',
   'img_02002.jpg',
   'img_04098.jpg',
   'img_03458.jpg',
   'img_02662.jpg',
   'img_07170.jpg',
   'img_02025.jpg',
   'img_05970.jpg',
   'img_07271.jpg',
   'img_04121.jpg',
   'img_02052.jpg',
   'img_05131.jpg',
   'img_01509.jpg',
   'img_01186.jpg',
   'img_03694.jpg',
   'img_04762.jpg',
   'img_02018.jpg',
   'img_07168.jpg',
   'img_03432.jpg',
   'img_06503.jpg',
   'img_01728.jpg',
   'img_05053.jpg',
   'img_07647.jpg',
   'img_01865.jpg',
   'img_07039.jpg',
   'img_02719.jpg',
   'img_01333.jpg',
   'img_06716.jpg',
   'img_04489.jpg',
   'img_06979.jpg',
   'img_06176.jpg',
   'img_04542.jpg',
   'img_07172.jpg',
   'img_01763.jpg',
   'img_03985.jpg',
   'img_07439.jpg',
   'img_03699.jpg',
   'img_04014.jpg',
   'img_02658.jpg',
   'img_06518.jpg',
   'img_05501.jpg',
   'img_07018.jpg',
   'img_02925.jpg',
   'img_02328.jpg',
   'img_01162.jpg',
   'img_03089.jpg',
   'img_01928.jpg',
   'img_06344.jpg',
   'im

Make submission (see last section of this notebook)...

## Improve with PseudoLabelling (val only) - classifier_model_v2_val_pseudo

In [65]:
def fit_pseudo(model, trn_conv_features, pseudo_conv_features, val_conv_features,
               trn_labels, pseudo_labels, val_labels,
               epochs = 1,
               trn_batch_size = 20,
               pseudo_batch_size = 12,
               val_batch_size = 32):    
    """ Based heavily on:
    https://shaoanlu.wordpress.com/2017/04/10/a-simple-pseudo-labeling-function-implementation-in-keras/
    https://github.com/shaoanlu/dogs-vs-cats-redux/blob/master/res50_incepV3_Xcept.ipynb
    """
    
    # determine when to re-shuffle index array (i.e. when every image has been trained on)
    num_batch_per_epoch_trn = int(trn_conv_features.shape[0]/trn_batch_size)
    num_batch_per_epoch_pseudo = int(pseudo_conv_features.shape[0]/pseudo_batch_size)
    num_batch_per_epoch_val = int(val_conv_features.shape[0]/val_batch_size)
    
#    print("Reshuffling (pseudo) val indexes after num_iterations = {}".format(num_batch_per_epoch_pseudo))
#    print("Reshuffling val indexes after num_iterations = {}".format(num_batch_per_epoch_val))
    
    #loss, acc = model.metrics_names
    
    all_comb_features = np.concatenate(
                (trn_conv_features,
                 val_conv_features), axis=0)
    all_comb_labels = np.concatenate(
            (trn_labels,
             val_labels), axis=0)
    

    for e in range(epochs):
        now = datetime.now().strftime("%Y%m%d_%H%M%S")
        print("\nRunning epoch {}: {}".format(e, now))

        # Shuffling index array on each new epoch.
        trn_start_batch_idxs = np.random.permutation(num_batch_per_epoch_trn)
        pseudo_start_batch_idxs = np.random.permutation(num_batch_per_epoch_pseudo)
        val_start_batch_idxs = np.random.permutation(num_batch_per_epoch_val)

        # i.e. for each batch...
        for idx in range(max(num_batch_per_epoch_trn, num_batch_per_epoch_pseudo, num_batch_per_epoch_val)):

            # Get an index number from the shuffled index array for the current loop i
            trn_start_batch_idx = trn_start_batch_idxs[idx%num_batch_per_epoch_trn]
            trn_start_data_idx = trn_start_batch_idx*trn_batch_size
            trn_start_data_idx_next = (trn_start_batch_idx+1)*trn_batch_size

            pseudo_start_batch_idx = pseudo_start_batch_idxs[idx%num_batch_per_epoch_pseudo]
            pseudo_start_data_idx = pseudo_start_batch_idx*pseudo_batch_size
            pseudo_start_data_idx_next = (pseudo_start_batch_idx+1)*pseudo_batch_size

            val_start_batch_idx = val_start_batch_idxs[idx%num_batch_per_epoch_val]
            val_start_data_idx = val_start_batch_idx*val_batch_size
            val_start_data_idx_next = (val_start_batch_idx+1)*val_batch_size

            comb_features = np.concatenate(
                (trn_conv_features[ trn_start_data_idx:trn_start_data_idx_next ],
                 val_conv_features[ pseudo_start_data_idx:pseudo_start_data_idx_next ]), axis=0)
            #print(comb_features.shape)

            comb_labels = np.concatenate(
                (trn_labels[ trn_start_data_idx:trn_start_data_idx_next ],
                 pseudo_val_labels[ pseudo_start_data_idx:pseudo_start_data_idx_next ]), axis=0)
            #print(comb_labels.shape)

            model.train_on_batch(comb_features, comb_labels)

            #print("Batch trn_loss = {}, trn_acc = {},".format(trn_loss, trn_acc))
            #print("Batch val_loss = {}, val_acc = {},".format(val_loss, val_acc))    

            # Reshuffle index array after model trained on the last mini-batch.
            if (idx+1)%num_batch_per_epoch_trn == 0:
                trn_start_batch_idxs = np.random.permutation(num_batch_per_epoch_trn)
            if (idx+1)%num_batch_per_epoch_pseudo == 0:
                pseudo_start_batch_idxs = np.random.permutation(num_batch_per_epoch_pseudo)
            if (idx+1)%num_batch_per_epoch_val == 0:
                val_start_batch_idxs = np.random.permutation(num_batch_per_epoch_val)

        trn_comb_loss, trn_comb_acc = model.evaluate(all_comb_features, all_comb_labels, verbose=0)
        val_loss, val_acc = model.evaluate(val_conv_features, val_labels, verbose=0)
        print("Epoch trn_loss (inc. pseudo) = {}, trn_acc (inc. pseudo) = {},".format(trn_comb_loss, trn_comb_acc))
        print("Epoch val_loss = {}, val_acc = {},".format(val_loss, val_acc))

In [62]:
pseudo_val_labels = classifier_model_v2.predict(val_conv_features, batch_size=38)

In [67]:
num_epochs = 2

K.set_value(classifier_model_v2.optimizer.lr, 0.01)

fit_pseudo(model=classifier_model_v2,
            trn_conv_features=trn_conv_features, 
            pseudo_conv_features=val_conv_features,
            val_conv_features=val_conv_features,

            trn_labels=trn_labels,
            pseudo_labels=pseudo_val_labels,
            val_labels=val_labels,

            epochs = num_epochs,

            trn_batch_size = 20,
            pseudo_batch_size = 12,
            val_batch_size = 32)


Running epoch 0: 20170601_151850
Epoch trn_loss (inc. pseudo) = 0.35689734492568326, trn_acc (inc. pseudo) = 0.8853587503309505,
Epoch val_loss = 0.4072040419829519, val_acc = 0.8789473703033046,

Running epoch 1: 20170601_151856
Epoch trn_loss (inc. pseudo) = 0.30502619769701356, trn_acc (inc. pseudo) = 0.9433412761450887,
Epoch val_loss = 0.34541360390813725, val_acc = 0.9342105275706241,


In [68]:
num_epochs = 2

K.set_value(classifier_model_v2.optimizer.lr, 0.0001)

fit_pseudo(model=classifier_model_v2,
            trn_conv_features=trn_conv_features, 
            pseudo_conv_features=val_conv_features,
            val_conv_features=val_conv_features,

            trn_labels=trn_labels,
            pseudo_labels=pseudo_val_labels,
            val_labels=val_labels,

            epochs = num_epochs,

            trn_batch_size = 20,
            pseudo_batch_size = 12,
            val_batch_size = 32)


Running epoch 0: 20170601_151906
Epoch trn_loss (inc. pseudo) = 0.28482189312131434, trn_acc (inc. pseudo) = 0.9436060365369341,
Epoch val_loss = 0.3288777831353639, val_acc = 0.9263157907285189,

Running epoch 1: 20170601_151912
Epoch trn_loss (inc. pseudo) = 0.275193781055153, trn_acc (inc. pseudo) = 0.947047921630924,
Epoch val_loss = 0.3257320291117618, val_acc = 0.9289473696758873,


NOT WORKING CORRECTLY (based on JH MixIterator method):

In [102]:
trn_batch_size = 20 # ensure num_trn_sample/trn_batch_size is int.
pseudo_val_batch_size = 12 # ensure num_trn_sample/trn_batch_size is int.
val_batch_size = 38 # ensure num_trn_sample/trn_batch_size is int.

def preprocess_input(x, data_format=None):
    # Based on function at https://github.com/fchollet/keras/blob/master/keras/applications/imagenet_utils.py
    # Altered for 3D input, assumes data_format='channels_last'
    
    """Preprocesses a tensor encoding a batch of images.
    # Arguments
        x: input Numpy tensor, 3D.
        data_format: data format of the image tensor.
    # Returns
        Preprocessed tensor.
    """
    # 'RGB'->'BGR'
    x = x[:, :, ::-1]
    # Zero-center by mean pixel
    x[:, :, 0] -= 103.939
    x[:, :, 1] -= 116.779
    x[:, :, 2] -= 123.68
    return x

from image_custom import ImageDataGenerator_custom
gen = ImageDataGenerator_custom(preprocessing_function=preprocess_input)
shuffle = True

trn_batches = gen.flow(trn_conv_features, trn_labels, 
                       shuffle=shuffle, 
                       batch_size=trn_batch_size)

pseudo_val_batches = gen.flow(val_conv_features, pseudo_val_labels, 
                              shuffle=shuffle, 
                              batch_size=pseudo_val_batch_size)

val_batches = gen.flow(val_conv_features, val_labels, 
                       shuffle=False, 
                       batch_size=val_batch_size)

In [104]:
#weighted_iter.reset()

K.set_value(classifier_model_v2.optimizer.lr, 0.01)
classifier_model_v2.fit_generator(generator=weighted_iter,
                                  steps_per_epoch=170,
                                  validation_data=(val_conv_features, val_labels),
                                  epochs=2)
# Ran 3 times

Epoch 1/2
170/170 [==============================] - 7s - loss: 0.6708 - acc: 0.7462 - val_loss: 5.5130 - val_acc: 0.4211

In [103]:
class WeightedIterator(object):
    # Based on JH `MixIterator` https://github.com/asmith26/courses/blob/master/deeplearning1/nbs/utils.py#L237-L262
    def __init__(self, iters):
        assert type(iters) is list, "Initialising a WeightedIterator requires one *list* argument."
        self.iters = iters
        self.n = sum([it.n for it in self.iters])

    def reset(self):
        for it in self.iters: it.reset()

    def __iter__(self):
        return self
    
    def __next__(self, *args, **kwargs):
        return self.next(*args, **kwargs)

    def next(self, *args, **kwargs):
        """For python 2.x.

        # Returns
            The next batch.
        """
        next_iters_batches = [it.next() for it in self.iters]
        next_batch_X = np.concatenate([next_iter_batch[0] for next_iter_batch in next_iters_batches])
        next_batch_Y = np.concatenate([next_iter_batch[1] for next_iter_batch in next_iters_batches])
        return next_batch_X, next_batch_Y

weighted_iter = WeightedIterator([val_batches, val_batches])

In [69]:
prediction_actual_mappings_val_pseudo = create_confusion_matrix_helper(classifier_model_v2)

Make submission (see last section of this notebook)...

## Improve with PseudoLabelling (val and test) - classifier_model_v2_val_and_test_pseudo

Create test_conv_features on resnet_base 

!!!WARNING: Don't re-run the below (takes a long time) - load save (see further below)

In [ ]:
test_stg1_batches = get_batches(TEST_STG1_DIR, batch_size=20, shuffle=False)

test1_conv_features = predict_generator_helper(resnet_base,
                                               test_stg1_batches,
                                               batch_size=20,
                                               num_batch_samples=len(test_stg1_batches.filenames))

In [94]:
test_stg2_batches = get_batches(TEST_STG2_DIR, batch_size=3, shuffle=False)

test2_conv_features = predict_generator_helper(resnet_base,
                                               test_stg2_batches,
                                               batch_size=3,
                                               num_batch_samples=len(test_stg2_batches.filenames))

Found 1000 images belonging to 1 classes.
Found 12153 images belonging to 1 classes.


In [101]:
def save_precomputed_data(filenames, conv_feats, labels, features_base_name="ResNet50_conv_feats/test_"):
    save_array(RESULTS_DIR+"/"+features_base_name+'filenames.dat', np.array(filenames))
    save_array(RESULTS_DIR+"/"+features_base_name+'conv_feats.dat', conv_feats)
    save_array(RESULTS_DIR+"/"+features_base_name+'labels.dat', np.array(labels))
    
save_precomputed_data(test_stg1_batches.filenames,
                      test1_conv_features,
                      one_hot(test_stg1_batches.classes),
                      features_base_name="ResNet50_conv_feats/test1_")
save_precomputed_data(test_stg2_batches.filenames,
                      test2_conv_features,
                      one_hot(test_stg2_batches.classes),
                      features_base_name="ResNet50_conv_feats/test2_")

In [74]:
test1_filenames, test1_conv_features, _ = load_precomputed_data("ResNet50_conv_feats/test1_")
test2_filenames, test2_conv_features, _ = load_precomputed_data("ResNet50_conv_feats/test2_")

assert len(test1_filenames) == 1000, "test1_filenames not as expected"
assert test1_conv_features.shape == (1000, 1, 1, 2048), "test1_conv_features not as expected"

assert len(test2_filenames) == 12153, "test2_filenames not as expected"
assert test2_conv_features.shape == (12153, 1, 1, 2048), "test2_conv_features not as expected"

test_filenames = test1_filenames+test2_filenames
test_conv_features = np.concatenate((test1_conv_features,
                                   test2_conv_features), axis=0)

assert len(test_filenames) == 13153, "test1_conv_features not as expected"
assert test_conv_features.shape == (13153, 1, 1, 2048), "test2_conv_features not as expected"

In [75]:
val_and_test_conv_features = np.concatenate(
                                (val_conv_features,
                                 test_conv_features), axis=0)

In [76]:
pseudo_val_and_test_predictions = classifier_model_v2.predict(val_and_test_conv_features,
                                                              batch_size=13)

In [79]:
num_epochs = 2

K.set_value(classifier_model_v2.optimizer.lr, 0.01)

fit_pseudo(model=classifier_model_v2,
            trn_conv_features=trn_conv_features, 
            pseudo_conv_features=val_and_test_conv_features,
            val_conv_features=val_conv_features,

            trn_labels=trn_labels,
            pseudo_labels=pseudo_val_and_test_predictions,
            val_labels=val_labels,

            epochs = num_epochs,

            trn_batch_size = 20,
            pseudo_batch_size = 13,
            val_batch_size = 32)


Running epoch 0: 20170601_152353
Epoch trn_loss (inc. pseudo) = 1.4833693182086516, trn_acc (inc. pseudo) = 0.4760391845379931,
Epoch val_loss = 1.4866855006468924, val_acc = 0.4710526315789474,

Running epoch 1: 20170601_152426
Epoch trn_loss (inc. pseudo) = 1.5362889632870667, trn_acc (inc. pseudo) = 0.4548583531903627,
Epoch val_loss = 1.530465276617753, val_acc = 0.45263157894736844,


In [ ]:
num_epochs = 2

K.set_value(classifier_model_v2.optimizer.lr, 0.01)

fit_pseudo(model=classifier_model_v2,
            trn_conv_features=trn_conv_features, 
            pseudo_conv_features=val_and_test_conv_features,
            val_conv_features=val_conv_features,

            trn_labels=trn_labels,
            pseudo_labels=pseudo_val_and_test_predictions,
            val_labels=val_labels,

            epochs = num_epochs,

            trn_batch_size = 20,
            pseudo_batch_size = 13,
            val_batch_size = 32)

In [80]:
prediction_actual_mappings_val_and_test_pseudo = create_confusion_matrix_helper(classifier_model_v2)

Hmmm... may need to change the network to handle the new data - all train for longer(??)

## Pseudo labelling on pseudo labelling - prediction_actual_mappings_val_and_test_pseudo_pseudo

In [ ]:
pseudo_pseudo_val_and_test_predictions = classifier_model_v2.predict(val_and_test_conv_features, batch_size=13)

In [ ]:
num_epochs = 5

K.set_value(classifier_model_v2.optimizer.lr, 0.01)

fit_pseudo(model=classifier_model_v2_val_and_test_pseudo,
            trn_conv_features=trn_conv_features, 
            pseudo_conv_features=val_and_test_conv_features,
            val_conv_features=val_conv_features,

            trn_labels=trn_labels,
            pseudo_labels=pseudo_pseudo_val_and_test_predictions,
            val_labels=val_labels,

            epochs = num_epochs,

            trn_batch_size = 20,
            pseudo_batch_size = 13,
            val_batch_size = 32)

In [ ]:
num_epochs = 2

K.set_value(classifier_model_v2.optimizer.lr, 0.0001)

fit_pseudo(model=classifier_model_v2,
            trn_conv_features=trn_conv_features, 
            pseudo_conv_features=val_and_test_conv_features,
            val_conv_features=val_conv_features,

            trn_labels=trn_labels,
            pseudo_labels=pseudo_pseudo_val_and_test_predictions,
            val_labels=val_labels,

            epochs = num_epochs,

            trn_batch_size = 20,
            pseudo_batch_size = 13,
            val_batch_size = 32)

In [ ]:
prediction_actual_mappings_val_and_test_pseudo_pseudo = create_confusion_matrix_helper(classifier_model_v2)

# 3. Generate Test Predictions

In [209]:
# EDIT MODEL_NAME AND RUN:
MODEL_NAME = "/classifier_model_v2_val_and_test_pseudo3/" # "/classifier_model_v2_base/", "/classifier_model_v2_val_pseudo/", "/classifier_model_v2_val_and_test_pseudo/" or "/pseudo_pseudo_val_and_test_predictions/"
preds = classifier_model_v2.predict(test_conv_features, batch_size=38)

assert preds.shape == (13153, 8), "The number in of predictions created is not equal to the number in test_stg2"

In [210]:
def do_clip(arr, mx, num_classes): return np.clip(arr, (1-mx)/(num_classes-1), mx)

In [211]:
subm = do_clip(preds, 0.82, 8)
subm_name = RESULTS_DIR+MODEL_NAME+'/subm.csv'
!mkdir $RESULTS_DIR$MODEL_NAME
classes = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']

In [212]:
import pandas as pd

submission = pd.DataFrame(np.squeeze(subm), columns=classes)
submission.insert(0, 'image', test_stg1_batches.filenames + test_stg2_batches.filenames)
submission.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,unknown/img_06681.jpg,0.456161,0.057849,0.025714,0.025714,0.112211,0.085467,0.043986,0.210023
1,unknown/img_06893.jpg,0.457073,0.052053,0.025714,0.025714,0.126368,0.085146,0.045586,0.200693
2,unknown/img_05393.jpg,0.449439,0.074635,0.025714,0.025714,0.126501,0.074723,0.040289,0.202568
3,unknown/img_01918.jpg,0.471267,0.052979,0.025714,0.025714,0.110380,0.083571,0.042715,0.208714
4,unknown/img_05568.jpg,0.459721,0.048871,0.027527,0.025714,0.131980,0.075072,0.044985,0.204815


In [213]:
submission.tail()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
13148,unknown/image_02894.jpg,0.448456,0.062886,0.025714,0.025714,0.115162,0.083881,0.044183,0.203973
13149,unknown/image_09323.jpg,0.456575,0.052428,0.025714,0.025714,0.130479,0.080032,0.042972,0.203950
13150,unknown/image_04388.jpg,0.457101,0.052510,0.026380,0.025714,0.132034,0.075152,0.043499,0.205660
13151,unknown/image_09519.jpg,0.459610,0.048895,0.027567,0.025714,0.133668,0.074266,0.044920,0.202179
13152,unknown/image_11250.jpg,0.451451,0.062804,0.025714,0.025714,0.114790,0.083147,0.042139,0.204714


In [214]:
submission.to_csv(subm_name, index=False)#, header=False)#, compression='gzip')

In [215]:
subm_name_stg1 = RESULTS_DIR+MODEL_NAME+'/subm_test_stg1.csv'
subm_name_stg2 = RESULTS_DIR+MODEL_NAME+'/subm_test_stg2.csv'
subm_nameFIXED = RESULTS_DIR+MODEL_NAME+'/submFIXED.csv'

# Fixed format for Kaggle submission
!head -n +1001 $subm_name | sed "s:^unknown/::g" > $subm_name_stg1
!tail -n +1002 $subm_name | sed "s:^unknown:test_stg2:g" > $subm_name_stg2
!cat $subm_name_stg1 $subm_name_stg2 > $subm_nameFIXED